# Topics


In [3]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
import json
import time
from tqdm import tqdm

In [4]:
sys.setrecursionlimit(1500)

In [5]:
with open("08-input", "r") as file:
    lines = file.readlines()
data_raw = "".join(lines)
# data_raw = [line.replace("\n", "") for line in lines]
# data_raw = "\n".join(data_raw)
data_raw

'...O.....0...............................p..k.....\nO.........o....w..T.........................p.....\n..................w..........oM...................\n.............................................Y....\no.............T...........................z.....pk\n.....................................z..Y....t.F..\n...........T..........................F.......Y...\n...................A............z...k..M..........\n....O.........j....w.....................M........\n..........w....T..................M..k............\n.............5.............................F.....t\n......................A.............F....E........\n.....................S.........A..................\n.P................................................\n........................A...E.............x...t...\n............j.........................t.........x.\n.......................j.........................x\n....................................E........c....\n.............P.......E............................\n...........

In [6]:
test_data_raw = r"""............
........0...
.....0......
.......0....
....0.......
......A.....
............
............
........A...
.........A..
............
............"""



def preprocess_data (data):
    # dtype='U10'
    
    rows = [list(row) for row in data.split("\n")]
    return np.array(rows, dtype='U10')



test_data = preprocess_data(test_data_raw)
# test_data2 = preprocess_data(test_data_raw2)
display(test_data)

array([['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '0', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '0', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '0', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '0', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', 'A', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', 'A', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']],
      dtype='<U10')

In [7]:
data = preprocess_data(data_raw)
data

array([['.', '.', '.', ..., '.', '.', '.'],
       ['O', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ...,
       ['.', '.', '.', ..., 'J', '.', '.'],
       ['8', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.']], dtype='<U10')

In [21]:
def solution (data, verbose=False):

    def isValid(shape : tuple, index : tuple):
        return (0 <= index[0] < shape[0] and
                0 <= index[1] < shape[1] 
                # 0 <= index[2] < shape[2]
                )

    antinodes = np.zeros_like(data, dtype=int)

    for freq in np.unique(data):
        if freq != '.':
            frequencies = np.argwhere(data == freq)
            for freq_pair in combinations(frequencies, 2):
                x = freq_pair[0][0] - freq_pair[1][0]
                y = freq_pair[0][1] - freq_pair[1][1]
                
                if isValid(antinodes.shape, (freq_pair[0][0] + x, freq_pair[0][1] + y) ):
                    antinodes[freq_pair[0][0] + x, freq_pair[0][1] + y] = 1
                if isValid(antinodes.shape, (freq_pair[1][0] - x, freq_pair[1][1] - y) ):
                    antinodes[freq_pair[1][0] - x, freq_pair[1][1] - y] = 1
    
    return antinodes.sum()



sol = solution(test_data, verbose=True)

print(sol)


[[0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0 0 0 1

In [41]:
sol = solution(data)
print(sol)

975671981569


# Part 2

In [ ]:
def solution2 (data, verbose=False):

    total = 0
    
    for test, eq in tqdm(data.items()):
           
        results = [eq[0]]
        for level in range(1, len(eq)):
            num = eq[level]
            results = [result*num   for result in results] + [result+num   for result in results] + [int(str(result)+str(num))   for result in results]
        
        if test in results:
            total += test
        

    return total



sol = solution2(test_data, verbose=True)
print(sol)
# print(dists)

# display(sum(sol))

    

100%|██████████| 9/9 [00:00<?, ?it/s]

11387


In [51]:
sol = solution2(data)
print(sol)

100%|██████████| 850/850 [00:05<00:00, 145.01it/s]

223472064194845
